# Day 03: API Design Patterns for Trading Systems
## Week 22: System Design

---

### Learning Objectives
- Understand RESTful API design principles for trading systems
- Implement WebSocket APIs for real-time market data streaming
- Design robust order management API patterns
- Master authentication and rate limiting strategies
- Build async API clients for high-throughput trading

### Topics Covered
1. RESTful API Design for Market Data
2. WebSocket Streaming APIs
3. Order Management API Patterns
4. Rate Limiting and Throttling
5. Authentication and Security
6. Request/Response Models with Pydantic
7. Async API Client Implementation
8. Error Handling and Retry Logic
9. API Versioning Strategies
10. Interview Questions

---

**Trading API Architecture Overview:**

```
┌─────────────────────────────────────────────────────────────────┐
│                     Trading System APIs                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌──────────────┐   ┌──────────────┐   ┌──────────────────┐    │
│  │   REST API   │   │  WebSocket   │   │   FIX Protocol   │    │
│  │              │   │              │   │                  │    │
│  │ • Orders     │   │ • Quotes     │   │ • Session Mgmt   │    │
│  │ • Positions  │   │ • Trades     │   │ • Order Flow     │    │
│  │ • Account    │   │ • Order Book │   │ • Executions     │    │
│  │ • History    │   │ • Fills      │   │ • Market Data    │    │
│  └──────────────┘   └──────────────┘   └──────────────────┘    │
│                                                                  │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │                    API Gateway                           │   │
│  │  • Rate Limiting  • Auth  • Routing  • Load Balancing   │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

## 1. Import Required Libraries

In [3]:
# Core Libraries
import asyncio
import json
import time
import hashlib
import hmac
import secrets
from datetime import datetime, timedelta
from typing import Optional, List, Dict, Any, Union
from enum import Enum
from dataclasses import dataclass, field
from collections import deque
import threading
from abc import ABC, abstractmethod
import uuid

# Data Handling
import pandas as pd
import numpy as np

# HTTP and Async
import requests
from urllib.parse import urlencode

# Pydantic for Data Validation
from pydantic import BaseModel, Field, validator, root_validator
from pydantic import ValidationError

# For demonstration purposes - these would be actual imports in production
# from fastapi import FastAPI, HTTPException, Depends, WebSocket, status
# from fastapi.security import APIKeyHeader
# import aiohttp
# import websockets

print("Libraries imported successfully!")
print(f"Python datetime: {datetime.now()}")

ModuleNotFoundError: No module named 'pydantic'

## 2. RESTful API Design for Market Data

### Key Design Principles for Trading APIs

1. **Resource-Oriented Design**: Structure around trading resources (orders, positions, quotes)
2. **Idempotency**: Same request produces same result (critical for order submission)
3. **Versioning**: Support API evolution without breaking clients
4. **Rate Limiting**: Protect system from abuse
5. **Low Latency**: Minimize response times for time-sensitive operations

### REST Endpoint Patterns

```
Trading API Endpoints:

Market Data:
  GET  /v1/quotes/{symbol}           - Current quote
  GET  /v1/candles/{symbol}          - Historical OHLCV
  GET  /v1/orderbook/{symbol}        - Order book snapshot
  GET  /v1/instruments               - Available instruments

Order Management:
  POST   /v1/orders                  - Submit new order
  GET    /v1/orders/{order_id}       - Get order status
  PUT    /v1/orders/{order_id}       - Modify order
  DELETE /v1/orders/{order_id}       - Cancel order
  GET    /v1/orders                  - List orders (with filters)

Account:
  GET  /v1/account/balance           - Account balance
  GET  /v1/account/positions         - Open positions
  GET  /v1/account/trades            - Trade history
```

In [ ]:
# ==============================================================================
# RESTful Trading API - Endpoint Definitions
# ==============================================================================

class HTTPMethod(Enum):
    """HTTP methods used in REST APIs."""
    GET = "GET"
    POST = "POST"
    PUT = "PUT"
    DELETE = "DELETE"
    PATCH = "PATCH"


@dataclass
class APIEndpoint:
    """Represents an API endpoint with its configuration."""
    path: str
    method: HTTPMethod
    description: str
    rate_limit: int = 100  # requests per minute
    requires_auth: bool = True
    idempotent: bool = True
    
    def __str__(self):
        return f"{self.method.value:6} {self.path}"


class TradingAPIDefinition:
    """
    Trading API endpoint definitions following REST best practices.
    
    Design Principles:
    - Use nouns for resources, verbs via HTTP methods
    - Hierarchical structure for related resources
    - Consistent naming conventions
    - Version in URL path
    """
    
    BASE_URL = "/api/v1"
    
    # Market Data Endpoints (read-only, high frequency)
    MARKET_DATA = {
        "get_quote": APIEndpoint(
            path="/quotes/{symbol}",
            method=HTTPMethod.GET,
            description="Get current quote for a symbol",
            rate_limit=300,  # Higher limit for market data
            requires_auth=True,
            idempotent=True
        ),
        "get_candles": APIEndpoint(
            path="/candles/{symbol}",
            method=HTTPMethod.GET,
            description="Get historical OHLCV candles",
            rate_limit=100,
            requires_auth=True,
            idempotent=True
        ),
        "get_orderbook": APIEndpoint(
            path="/orderbook/{symbol}",
            method=HTTPMethod.GET,
            description="Get order book snapshot",
            rate_limit=200,
            requires_auth=True,
            idempotent=True
        ),
        "list_instruments": APIEndpoint(
            path="/instruments",
            method=HTTPMethod.GET,
            description="List available trading instruments",
            rate_limit=10,
            requires_auth=True,
            idempotent=True
        ),
    }
    
    # Order Management Endpoints (write operations, critical)
    ORDER_MANAGEMENT = {
        "submit_order": APIEndpoint(
            path="/orders",
            method=HTTPMethod.POST,
            description="Submit a new order",
            rate_limit=50,
            requires_auth=True,
            idempotent=False  # Use idempotency key in header
        ),
        "get_order": APIEndpoint(
            path="/orders/{order_id}",
            method=HTTPMethod.GET,
            description="Get order by ID",
            rate_limit=100,
            requires_auth=True,
            idempotent=True
        ),
        "modify_order": APIEndpoint(
            path="/orders/{order_id}",
            method=HTTPMethod.PUT,
            description="Modify an existing order",
            rate_limit=50,
            requires_auth=True,
            idempotent=True
        ),
        "cancel_order": APIEndpoint(
            path="/orders/{order_id}",
            method=HTTPMethod.DELETE,
            description="Cancel an order",
            rate_limit=50,
            requires_auth=True,
            idempotent=True  # Canceling twice has same effect
        ),
        "list_orders": APIEndpoint(
            path="/orders",
            method=HTTPMethod.GET,
            description="List orders with filters",
            rate_limit=30,
            requires_auth=True,
            idempotent=True
        ),
    }
    
    # Account Endpoints
    ACCOUNT = {
        "get_balance": APIEndpoint(
            path="/account/balance",
            method=HTTPMethod.GET,
            description="Get account balance",
            rate_limit=60,
            requires_auth=True,
            idempotent=True
        ),
        "get_positions": APIEndpoint(
            path="/account/positions",
            method=HTTPMethod.GET,
            description="Get open positions",
            rate_limit=60,
            requires_auth=True,
            idempotent=True
        ),
        "get_trades": APIEndpoint(
            path="/account/trades",
            method=HTTPMethod.GET,
            description="Get trade history",
            rate_limit=30,
            requires_auth=True,
            idempotent=True
        ),
    }
    
    @classmethod
    def print_all_endpoints(cls):
        """Print all API endpoints in a formatted way."""
        print("=" * 60)
        print("TRADING API ENDPOINTS (v1)")
        print("=" * 60)
        
        categories = [
            ("MARKET DATA", cls.MARKET_DATA),
            ("ORDER MANAGEMENT", cls.ORDER_MANAGEMENT),
            ("ACCOUNT", cls.ACCOUNT),
        ]
        
        for category_name, endpoints in categories:
            print(f"\n{category_name}:")
            print("-" * 40)
            for name, endpoint in endpoints.items():
                print(f"  {endpoint}")
                print(f"    Rate Limit: {endpoint.rate_limit}/min")
        
        print("\n" + "=" * 60)


# Display API endpoints
TradingAPIDefinition.print_all_endpoints()

In [ ]:
# ==============================================================================
# Simulated REST API Client for Market Data
# ==============================================================================

class MockMarketDataAPI:
    """
    Simulated market data API for demonstration.
    In production, this would make actual HTTP requests.
    """
    
    def __init__(self, base_url: str = "https://api.trading.example.com"):
        self.base_url = base_url
        self._mock_data = self._generate_mock_data()
    
    def _generate_mock_data(self) -> Dict:
        """Generate mock market data."""
        symbols = ["AAPL", "GOOGL", "MSFT", "TSLA", "SPY"]
        quotes = {}
        
        np.random.seed(42)
        for symbol in symbols:
            base_price = np.random.uniform(100, 500)
            quotes[symbol] = {
                "symbol": symbol,
                "bid": round(base_price * 0.999, 2),
                "ask": round(base_price * 1.001, 2),
                "bid_size": np.random.randint(100, 1000),
                "ask_size": np.random.randint(100, 1000),
                "last": round(base_price, 2),
                "volume": np.random.randint(1000000, 10000000),
                "timestamp": datetime.now().isoformat()
            }
        
        return {"quotes": quotes}
    
    def get_quote(self, symbol: str) -> Dict:
        """
        GET /v1/quotes/{symbol}
        
        Returns current quote for a symbol.
        """
        if symbol not in self._mock_data["quotes"]:
            return {
                "error": {
                    "code": "SYMBOL_NOT_FOUND",
                    "message": f"Symbol {symbol} not found",
                    "status": 404
                }
            }
        
        # Simulate response structure
        return {
            "data": self._mock_data["quotes"][symbol],
            "meta": {
                "request_id": str(uuid.uuid4()),
                "timestamp": datetime.now().isoformat(),
                "latency_ms": np.random.uniform(1, 5)
            }
        }
    
    def get_candles(
        self,
        symbol: str,
        interval: str = "1h",
        start: str = None,
        end: str = None,
        limit: int = 100
    ) -> Dict:
        """
        GET /v1/candles/{symbol}?interval={interval}&start={start}&end={end}&limit={limit}
        
        Returns historical OHLCV data.
        """
        # Generate mock candles
        np.random.seed(hash(symbol) % 2**32)
        base_price = np.random.uniform(100, 500)
        
        candles = []
        current_time = datetime.now()
        
        for i in range(limit):
            timestamp = current_time - timedelta(hours=i)
            open_price = base_price * (1 + np.random.uniform(-0.02, 0.02))
            close_price = open_price * (1 + np.random.uniform(-0.01, 0.01))
            high_price = max(open_price, close_price) * (1 + np.random.uniform(0, 0.01))
            low_price = min(open_price, close_price) * (1 - np.random.uniform(0, 0.01))
            
            candles.append({
                "timestamp": timestamp.isoformat(),
                "open": round(open_price, 2),
                "high": round(high_price, 2),
                "low": round(low_price, 2),
                "close": round(close_price, 2),
                "volume": np.random.randint(100000, 1000000)
            })
            base_price = close_price
        
        return {
            "data": {
                "symbol": symbol,
                "interval": interval,
                "candles": candles[::-1]  # Chronological order
            },
            "meta": {
                "request_id": str(uuid.uuid4()),
                "timestamp": datetime.now().isoformat(),
                "count": len(candles)
            }
        }
    
    def get_orderbook(self, symbol: str, depth: int = 10) -> Dict:
        """
        GET /v1/orderbook/{symbol}?depth={depth}
        
        Returns order book snapshot.
        """
        if symbol not in self._mock_data["quotes"]:
            return {"error": {"code": "SYMBOL_NOT_FOUND", "status": 404}}
        
        quote = self._mock_data["quotes"][symbol]
        mid_price = (quote["bid"] + quote["ask"]) / 2
        
        bids = []
        asks = []
        
        for i in range(depth):
            bid_price = mid_price * (1 - 0.0001 * (i + 1))
            ask_price = mid_price * (1 + 0.0001 * (i + 1))
            
            bids.append({
                "price": round(bid_price, 2),
                "size": np.random.randint(100, 5000),
                "count": np.random.randint(1, 10)
            })
            asks.append({
                "price": round(ask_price, 2),
                "size": np.random.randint(100, 5000),
                "count": np.random.randint(1, 10)
            })
        
        return {
            "data": {
                "symbol": symbol,
                "bids": bids,
                "asks": asks,
                "timestamp": datetime.now().isoformat()
            },
            "meta": {
                "request_id": str(uuid.uuid4()),
                "depth": depth
            }
        }


# Demonstrate API usage
api = MockMarketDataAPI()

print("=" * 60)
print("MARKET DATA API EXAMPLES")
print("=" * 60)

# Get quote
print("\n1. GET /v1/quotes/AAPL")
quote_response = api.get_quote("AAPL")
print(json.dumps(quote_response, indent=2))

# Get candles (limited output)
print("\n2. GET /v1/candles/AAPL?interval=1h&limit=5")
candles_response = api.get_candles("AAPL", interval="1h", limit=5)
print(json.dumps(candles_response, indent=2))

# Get orderbook (limited output)
print("\n3. GET /v1/orderbook/AAPL?depth=3")
orderbook_response = api.get_orderbook("AAPL", depth=3)
print(json.dumps(orderbook_response, indent=2))

## 3. WebSocket API for Real-Time Price Streams

### WebSocket vs REST for Trading

| Feature | REST | WebSocket |
|---------|------|-----------|
| Connection | New connection per request | Persistent connection |
| Latency | Higher (HTTP overhead) | Lower (direct messaging) |
| Use Case | On-demand queries | Real-time streaming |
| Server Push | Not supported | Native support |
| Bandwidth | Higher (headers per request) | Lower (minimal framing) |

### WebSocket Message Types

```
Client → Server (Subscribe):
{
  "action": "subscribe",
  "channel": "quotes",
  "symbols": ["AAPL", "GOOGL"]
}

Server → Client (Quote Update):
{
  "channel": "quotes",
  "symbol": "AAPL",
  "data": {
    "bid": 150.25,
    "ask": 150.27,
    "timestamp": "2024-01-15T10:30:00.123Z"
  }
}
```

In [ ]:
# ==============================================================================
# WebSocket Message Protocol Definition
# ==============================================================================

class WebSocketAction(Enum):
    """Actions that can be sent by the client."""
    SUBSCRIBE = "subscribe"
    UNSUBSCRIBE = "unsubscribe"
    PING = "ping"
    AUTHENTICATE = "authenticate"


class WebSocketChannel(Enum):
    """Available streaming channels."""
    QUOTES = "quotes"           # Real-time quotes
    TRADES = "trades"           # Trade executions (market)
    ORDERBOOK = "orderbook"     # Order book updates
    CANDLES = "candles"         # OHLCV updates
    ORDERS = "orders"           # User's order updates
    FILLS = "fills"             # User's fill notifications
    POSITIONS = "positions"     # Position changes


@dataclass
class WebSocketMessage:
    """Base class for WebSocket messages."""
    action: str
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())
    request_id: str = field(default_factory=lambda: str(uuid.uuid4()))


class WebSocketProtocol:
    """
    WebSocket protocol specification for trading data.
    
    Connection Flow:
    1. Connect to wss://ws.trading.example.com/v1/stream
    2. Send authentication message
    3. Subscribe to channels
    4. Receive streaming data
    5. Handle heartbeats (ping/pong)
    """
    
    @staticmethod
    def create_auth_message(api_key: str, signature: str) -> Dict:
        """Create authentication message."""
        return {
            "action": "authenticate",
            "api_key": api_key,
            "signature": signature,
            "timestamp": datetime.now().isoformat()
        }
    
    @staticmethod
    def create_subscribe_message(
        channel: WebSocketChannel,
        symbols: List[str] = None,
        **kwargs
    ) -> Dict:
        """
        Create subscription message.
        
        Examples:
            subscribe_message("quotes", ["AAPL", "GOOGL"])
            subscribe_message("orderbook", ["AAPL"], depth=10)
        """
        message = {
            "action": "subscribe",
            "channel": channel.value,
            "request_id": str(uuid.uuid4()),
            "timestamp": datetime.now().isoformat()
        }
        
        if symbols:
            message["symbols"] = symbols
        
        message.update(kwargs)
        return message
    
    @staticmethod
    def create_unsubscribe_message(
        channel: WebSocketChannel,
        symbols: List[str] = None
    ) -> Dict:
        """Create unsubscription message."""
        message = {
            "action": "unsubscribe",
            "channel": channel.value,
            "request_id": str(uuid.uuid4()),
            "timestamp": datetime.now().isoformat()
        }
        
        if symbols:
            message["symbols"] = symbols
        
        return message
    
    @staticmethod
    def parse_server_message(raw_message: str) -> Dict:
        """Parse incoming server message."""
        try:
            message = json.loads(raw_message)
            return {
                "success": True,
                "data": message
            }
        except json.JSONDecodeError as e:
            return {
                "success": False,
                "error": f"Failed to parse message: {e}"
            }


# Demonstrate WebSocket message creation
print("=" * 60)
print("WEBSOCKET MESSAGE EXAMPLES")
print("=" * 60)

# Authentication
print("\n1. Authentication Message:")
auth_msg = WebSocketProtocol.create_auth_message(
    api_key="your-api-key",
    signature="hmac-signature"
)
print(json.dumps(auth_msg, indent=2))

# Subscribe to quotes
print("\n2. Subscribe to Quotes:")
subscribe_msg = WebSocketProtocol.create_subscribe_message(
    channel=WebSocketChannel.QUOTES,
    symbols=["AAPL", "GOOGL", "MSFT"]
)
print(json.dumps(subscribe_msg, indent=2))

# Subscribe to orderbook with depth
print("\n3. Subscribe to Order Book:")
orderbook_msg = WebSocketProtocol.create_subscribe_message(
    channel=WebSocketChannel.ORDERBOOK,
    symbols=["AAPL"],
    depth=20
)
print(json.dumps(orderbook_msg, indent=2))

# Unsubscribe
print("\n4. Unsubscribe Message:")
unsub_msg = WebSocketProtocol.create_unsubscribe_message(
    channel=WebSocketChannel.QUOTES,
    symbols=["GOOGL"]
)
print(json.dumps(unsub_msg, indent=2))

In [ ]:
# ==============================================================================
# WebSocket Client Implementation (Simulated)
# ==============================================================================

class SimulatedWebSocketClient:
    """
    Simulated WebSocket client for demonstration.
    
    In production, this would use the `websockets` library:
    
    ```python
    import websockets
    
    async def connect():
        async with websockets.connect('wss://ws.exchange.com/stream') as ws:
            await ws.send(json.dumps(subscribe_message))
            async for message in ws:
                process_message(message)
    ```
    """
    
    def __init__(self, url: str = "wss://ws.trading.example.com/v1/stream"):
        self.url = url
        self.connected = False
        self.authenticated = False
        self.subscriptions: Dict[str, List[str]] = {}
        self._callbacks: Dict[str, callable] = {}
        self._message_queue: deque = deque(maxlen=1000)
    
    def connect(self) -> Dict:
        """Establish WebSocket connection."""
        self.connected = True
        return {
            "status": "connected",
            "url": self.url,
            "timestamp": datetime.now().isoformat()
        }
    
    def authenticate(self, api_key: str, secret: str) -> Dict:
        """Authenticate the connection."""
        if not self.connected:
            return {"error": "Not connected"}
        
        # In production: create HMAC signature
        timestamp = str(int(time.time() * 1000))
        message = f"{timestamp}authenticate"
        signature = hmac.new(
            secret.encode(),
            message.encode(),
            hashlib.sha256
        ).hexdigest()
        
        self.authenticated = True
        return {
            "status": "authenticated",
            "api_key": api_key[:8] + "...",
            "timestamp": datetime.now().isoformat()
        }
    
    def subscribe(self, channel: str, symbols: List[str]) -> Dict:
        """Subscribe to a channel."""
        if not self.authenticated:
            return {"error": "Not authenticated"}
        
        if channel not in self.subscriptions:
            self.subscriptions[channel] = []
        
        for symbol in symbols:
            if symbol not in self.subscriptions[channel]:
                self.subscriptions[channel].append(symbol)
        
        return {
            "status": "subscribed",
            "channel": channel,
            "symbols": self.subscriptions[channel],
            "timestamp": datetime.now().isoformat()
        }
    
    def unsubscribe(self, channel: str, symbols: List[str] = None) -> Dict:
        """Unsubscribe from a channel."""
        if channel not in self.subscriptions:
            return {"error": f"Not subscribed to {channel}"}
        
        if symbols:
            for symbol in symbols:
                if symbol in self.subscriptions[channel]:
                    self.subscriptions[channel].remove(symbol)
        else:
            self.subscriptions[channel] = []
        
        return {
            "status": "unsubscribed",
            "channel": channel,
            "remaining": self.subscriptions[channel]
        }
    
    def simulate_incoming_messages(self, count: int = 5) -> List[Dict]:
        """Simulate receiving market data messages."""
        messages = []
        
        for channel, symbols in self.subscriptions.items():
            for symbol in symbols:
                for _ in range(count):
                    if channel == "quotes":
                        msg = self._generate_quote_message(symbol)
                    elif channel == "trades":
                        msg = self._generate_trade_message(symbol)
                    elif channel == "orderbook":
                        msg = self._generate_orderbook_update(symbol)
                    else:
                        msg = {"channel": channel, "symbol": symbol}
                    
                    messages.append(msg)
        
        return messages
    
    def _generate_quote_message(self, symbol: str) -> Dict:
        """Generate simulated quote update."""
        base_price = 150.0 + hash(symbol) % 100
        return {
            "channel": "quotes",
            "type": "update",
            "symbol": symbol,
            "data": {
                "bid": round(base_price * (1 + np.random.uniform(-0.001, 0)), 2),
                "ask": round(base_price * (1 + np.random.uniform(0, 0.001)), 2),
                "bid_size": np.random.randint(100, 1000),
                "ask_size": np.random.randint(100, 1000),
                "timestamp": datetime.now().isoformat()
            }
        }
    
    def _generate_trade_message(self, symbol: str) -> Dict:
        """Generate simulated trade message."""
        base_price = 150.0 + hash(symbol) % 100
        return {
            "channel": "trades",
            "type": "execution",
            "symbol": symbol,
            "data": {
                "price": round(base_price * (1 + np.random.uniform(-0.001, 0.001)), 2),
                "size": np.random.randint(1, 100),
                "side": np.random.choice(["buy", "sell"]),
                "trade_id": str(uuid.uuid4())[:8],
                "timestamp": datetime.now().isoformat()
            }
        }
    
    def _generate_orderbook_update(self, symbol: str) -> Dict:
        """Generate simulated order book update."""
        base_price = 150.0 + hash(symbol) % 100
        return {
            "channel": "orderbook",
            "type": "delta",
            "symbol": symbol,
            "data": {
                "bids": [
                    {"price": round(base_price - 0.01 * i, 2), 
                     "size": np.random.randint(100, 1000)}
                    for i in range(3)
                ],
                "asks": [
                    {"price": round(base_price + 0.01 * i, 2),
                     "size": np.random.randint(100, 1000)}
                    for i in range(3)
                ],
                "timestamp": datetime.now().isoformat()
            }
        }
    
    def disconnect(self) -> Dict:
        """Close WebSocket connection."""
        self.connected = False
        self.authenticated = False
        self.subscriptions = {}
        return {"status": "disconnected"}


# Demonstrate WebSocket client usage
print("=" * 60)
print("WEBSOCKET CLIENT DEMONSTRATION")
print("=" * 60)

# Create client and connect
ws_client = SimulatedWebSocketClient()

print("\n1. Connecting...")
print(ws_client.connect())

print("\n2. Authenticating...")
print(ws_client.authenticate("api-key-123", "secret-456"))

print("\n3. Subscribing to quotes...")
print(ws_client.subscribe("quotes", ["AAPL", "GOOGL"]))

print("\n4. Subscribing to trades...")
print(ws_client.subscribe("trades", ["AAPL"]))

print("\n5. Simulated incoming messages:")
messages = ws_client.simulate_incoming_messages(count=2)
for i, msg in enumerate(messages[:4], 1):
    print(f"\n  Message {i}:")
    print(f"  {json.dumps(msg, indent=4)}")

print("\n6. Disconnecting...")
print(ws_client.disconnect())

## 4. Order Management API Patterns

### Critical Design Considerations

1. **Idempotency**: Use idempotency keys to prevent duplicate orders
2. **Atomic Operations**: Order submission should be all-or-nothing
3. **State Machine**: Orders follow a defined lifecycle
4. **Audit Trail**: Every state change must be logged
5. **Validation**: Strict input validation before processing

### Order Lifecycle State Machine

```
                    ┌──────────┐
                    │  NEW     │
                    └────┬─────┘
                         │
         ┌───────────────┼───────────────┐
         │               │               │
         ▼               ▼               ▼
    ┌─────────┐    ┌──────────┐    ┌──────────┐
    │ PENDING │    │ REJECTED │    │ CANCELED │
    └────┬────┘    └──────────┘    └──────────┘
         │                               ▲
         │                               │
         ▼                               │
    ┌─────────────┐                      │
    │ OPEN/ACTIVE │──────────────────────┤
    └──────┬──────┘                      │
           │                             │
    ┌──────┴──────┐                      │
    │             │                      │
    ▼             ▼                      │
┌────────┐   ┌──────────────┐           │
│ FILLED │   │ PARTIALLY    │───────────┘
└────────┘   │ FILLED       │
             └──────────────┘
```

In [ ]:
# ==============================================================================
# Order Management API - Data Models
# ==============================================================================

class OrderSide(str, Enum):
    """Order side."""
    BUY = "buy"
    SELL = "sell"


class OrderType(str, Enum):
    """Order type."""
    MARKET = "market"
    LIMIT = "limit"
    STOP = "stop"
    STOP_LIMIT = "stop_limit"


class TimeInForce(str, Enum):
    """Time in force."""
    GTC = "gtc"        # Good Till Canceled
    DAY = "day"        # Day order
    IOC = "ioc"        # Immediate or Cancel
    FOK = "fok"        # Fill or Kill
    GTD = "gtd"        # Good Till Date


class OrderStatus(str, Enum):
    """Order status."""
    NEW = "new"
    PENDING = "pending"
    OPEN = "open"
    PARTIALLY_FILLED = "partially_filled"
    FILLED = "filled"
    CANCELED = "canceled"
    REJECTED = "rejected"
    EXPIRED = "expired"


# Pydantic Models for API Request/Response
class OrderRequest(BaseModel):
    """
    Order submission request model.
    
    POST /v1/orders
    """
    symbol: str = Field(..., min_length=1, max_length=20, description="Trading symbol")
    side: OrderSide = Field(..., description="Buy or sell")
    order_type: OrderType = Field(..., description="Order type")
    quantity: float = Field(..., gt=0, description="Order quantity")
    price: Optional[float] = Field(None, gt=0, description="Limit price (required for limit orders)")
    stop_price: Optional[float] = Field(None, gt=0, description="Stop trigger price")
    time_in_force: TimeInForce = Field(default=TimeInForce.GTC)
    client_order_id: Optional[str] = Field(None, max_length=50, description="Client-provided order ID")
    
    @validator('price')
    def validate_price_for_limit(cls, v, values):
        """Validate that price is provided for limit orders."""
        if values.get('order_type') in [OrderType.LIMIT, OrderType.STOP_LIMIT]:
            if v is None:
                raise ValueError("Price is required for limit orders")
        return v
    
    @validator('stop_price')
    def validate_stop_price(cls, v, values):
        """Validate that stop_price is provided for stop orders."""
        if values.get('order_type') in [OrderType.STOP, OrderType.STOP_LIMIT]:
            if v is None:
                raise ValueError("Stop price is required for stop orders")
        return v
    
    class Config:
        use_enum_values = True


class OrderModifyRequest(BaseModel):
    """
    Order modification request model.
    
    PUT /v1/orders/{order_id}
    """
    quantity: Optional[float] = Field(None, gt=0, description="New quantity")
    price: Optional[float] = Field(None, gt=0, description="New price")
    stop_price: Optional[float] = Field(None, gt=0, description="New stop price")
    
    @root_validator
    def check_at_least_one_field(cls, values):
        """Ensure at least one field is being modified."""
        if not any([values.get('quantity'), values.get('price'), values.get('stop_price')]):
            raise ValueError("At least one field must be provided for modification")
        return values


class Fill(BaseModel):
    """Represents a fill/execution."""
    fill_id: str
    price: float
    quantity: float
    timestamp: datetime
    fee: float = 0.0
    fee_currency: str = "USD"


class OrderResponse(BaseModel):
    """
    Order response model.
    
    Returned by GET/POST/PUT /v1/orders
    """
    order_id: str
    client_order_id: Optional[str]
    symbol: str
    side: OrderSide
    order_type: OrderType
    quantity: float
    filled_quantity: float = 0.0
    remaining_quantity: float
    price: Optional[float]
    stop_price: Optional[float]
    average_fill_price: Optional[float]
    status: OrderStatus
    time_in_force: TimeInForce
    fills: List[Fill] = []
    created_at: datetime
    updated_at: datetime
    
    class Config:
        use_enum_values = True


# Demonstrate validation
print("=" * 60)
print("ORDER REQUEST VALIDATION")
print("=" * 60)

# Valid limit order
print("\n1. Valid Limit Order:")
try:
    valid_order = OrderRequest(
        symbol="AAPL",
        side=OrderSide.BUY,
        order_type=OrderType.LIMIT,
        quantity=100,
        price=150.50,
        time_in_force=TimeInForce.GTC,
        client_order_id="my-order-001"
    )
    print(f"  ✓ Valid: {valid_order.dict()}")
except ValidationError as e:
    print(f"  ✗ Error: {e}")

# Invalid limit order (missing price)
print("\n2. Invalid Limit Order (missing price):")
try:
    invalid_order = OrderRequest(
        symbol="AAPL",
        side=OrderSide.BUY,
        order_type=OrderType.LIMIT,
        quantity=100,
    )
except ValidationError as e:
    print(f"  ✗ Validation Error: {e.errors()[0]['msg']}")

# Valid market order
print("\n3. Valid Market Order:")
try:
    market_order = OrderRequest(
        symbol="GOOGL",
        side=OrderSide.SELL,
        order_type=OrderType.MARKET,
        quantity=50,
    )
    print(f"  ✓ Valid: {market_order.dict()}")
except ValidationError as e:
    print(f"  ✗ Error: {e}")

# Valid stop-limit order
print("\n4. Valid Stop-Limit Order:")
try:
    stop_limit_order = OrderRequest(
        symbol="TSLA",
        side=OrderSide.SELL,
        order_type=OrderType.STOP_LIMIT,
        quantity=25,
        price=200.00,
        stop_price=210.00,
        time_in_force=TimeInForce.DAY
    )
    print(f"  ✓ Valid: {stop_limit_order.dict()}")
except ValidationError as e:
    print(f"  ✗ Error: {e}")

In [ ]:
# ==============================================================================
# Order Management API - Implementation with Idempotency
# ==============================================================================

class OrderManagementAPI:
    """
    Order Management API with idempotency support.
    
    Key Features:
    - Idempotency key handling to prevent duplicate orders
    - Order state machine management
    - Audit logging for all operations
    """
    
    def __init__(self):
        self._orders: Dict[str, Dict] = {}
        self._idempotency_cache: Dict[str, str] = {}  # idempotency_key -> order_id
        self._audit_log: List[Dict] = []
    
    def _generate_order_id(self) -> str:
        """Generate unique order ID."""
        return f"ORD-{uuid.uuid4().hex[:12].upper()}"
    
    def _log_audit(self, action: str, order_id: str, details: Dict):
        """Log audit entry."""
        self._audit_log.append({
            "timestamp": datetime.now().isoformat(),
            "action": action,
            "order_id": order_id,
            "details": details
        })
    
    def submit_order(
        self,
        order_request: OrderRequest,
        idempotency_key: Optional[str] = None
    ) -> Dict:
        """
        POST /v1/orders
        
        Submit a new order with optional idempotency key.
        
        Headers:
            X-Idempotency-Key: Optional unique key for request deduplication
        
        Returns:
            OrderResponse or error
        """
        # Check idempotency
        if idempotency_key and idempotency_key in self._idempotency_cache:
            existing_order_id = self._idempotency_cache[idempotency_key]
            return {
                "status": "success",
                "data": self._orders[existing_order_id],
                "meta": {
                    "idempotent_replay": True,
                    "message": "Order already submitted with this idempotency key"
                }
            }
        
        # Generate order ID
        order_id = self._generate_order_id()
        
        # Create order record
        now = datetime.now()
        order = {
            "order_id": order_id,
            "client_order_id": order_request.client_order_id,
            "symbol": order_request.symbol,
            "side": order_request.side.value if isinstance(order_request.side, Enum) else order_request.side,
            "order_type": order_request.order_type.value if isinstance(order_request.order_type, Enum) else order_request.order_type,
            "quantity": order_request.quantity,
            "filled_quantity": 0.0,
            "remaining_quantity": order_request.quantity,
            "price": order_request.price,
            "stop_price": order_request.stop_price,
            "average_fill_price": None,
            "status": OrderStatus.NEW.value,
            "time_in_force": order_request.time_in_force.value if isinstance(order_request.time_in_force, Enum) else order_request.time_in_force,
            "fills": [],
            "created_at": now.isoformat(),
            "updated_at": now.isoformat()
        }
        
        # Store order
        self._orders[order_id] = order
        
        # Store idempotency mapping
        if idempotency_key:
            self._idempotency_cache[idempotency_key] = order_id
        
        # Audit log
        self._log_audit("ORDER_SUBMITTED", order_id, {"request": order_request.dict()})
        
        # Simulate order acceptance (in reality, this would go to matching engine)
        order["status"] = OrderStatus.OPEN.value
        order["updated_at"] = datetime.now().isoformat()
        
        return {
            "status": "success",
            "data": order,
            "meta": {
                "request_id": str(uuid.uuid4()),
                "idempotent_replay": False
            }
        }
    
    def get_order(self, order_id: str) -> Dict:
        """
        GET /v1/orders/{order_id}
        
        Get order by ID.
        """
        if order_id not in self._orders:
            return {
                "status": "error",
                "error": {
                    "code": "ORDER_NOT_FOUND",
                    "message": f"Order {order_id} not found",
                    "status": 404
                }
            }
        
        return {
            "status": "success",
            "data": self._orders[order_id]
        }
    
    def modify_order(self, order_id: str, modify_request: OrderModifyRequest) -> Dict:
        """
        PUT /v1/orders/{order_id}
        
        Modify an existing order.
        """
        if order_id not in self._orders:
            return {
                "status": "error",
                "error": {
                    "code": "ORDER_NOT_FOUND",
                    "message": f"Order {order_id} not found",
                    "status": 404
                }
            }
        
        order = self._orders[order_id]
        
        # Check if order can be modified
        if order["status"] not in [OrderStatus.NEW.value, OrderStatus.OPEN.value]:
            return {
                "status": "error",
                "error": {
                    "code": "ORDER_NOT_MODIFIABLE",
                    "message": f"Order in status {order['status']} cannot be modified",
                    "status": 400
                }
            }
        
        # Apply modifications
        original_values = {}
        if modify_request.quantity:
            original_values["quantity"] = order["quantity"]
            order["quantity"] = modify_request.quantity
            order["remaining_quantity"] = modify_request.quantity - order["filled_quantity"]
        
        if modify_request.price:
            original_values["price"] = order["price"]
            order["price"] = modify_request.price
        
        if modify_request.stop_price:
            original_values["stop_price"] = order["stop_price"]
            order["stop_price"] = modify_request.stop_price
        
        order["updated_at"] = datetime.now().isoformat()
        
        # Audit log
        self._log_audit("ORDER_MODIFIED", order_id, {
            "original": original_values,
            "new": modify_request.dict(exclude_none=True)
        })
        
        return {
            "status": "success",
            "data": order
        }
    
    def cancel_order(self, order_id: str) -> Dict:
        """
        DELETE /v1/orders/{order_id}
        
        Cancel an order. This operation is idempotent.
        """
        if order_id not in self._orders:
            return {
                "status": "error",
                "error": {
                    "code": "ORDER_NOT_FOUND",
                    "message": f"Order {order_id} not found",
                    "status": 404
                }
            }
        
        order = self._orders[order_id]
        
        # Idempotent: if already canceled, return success
        if order["status"] == OrderStatus.CANCELED.value:
            return {
                "status": "success",
                "data": order,
                "meta": {"already_canceled": True}
            }
        
        # Check if order can be canceled
        if order["status"] in [OrderStatus.FILLED.value, OrderStatus.REJECTED.value]:
            return {
                "status": "error",
                "error": {
                    "code": "ORDER_NOT_CANCELABLE",
                    "message": f"Order in status {order['status']} cannot be canceled",
                    "status": 400
                }
            }
        
        # Cancel the order
        order["status"] = OrderStatus.CANCELED.value
        order["updated_at"] = datetime.now().isoformat()
        
        # Audit log
        self._log_audit("ORDER_CANCELED", order_id, {})
        
        return {
            "status": "success",
            "data": order,
            "meta": {"already_canceled": False}
        }
    
    def list_orders(
        self,
        status: Optional[str] = None,
        symbol: Optional[str] = None,
        limit: int = 100
    ) -> Dict:
        """
        GET /v1/orders?status={status}&symbol={symbol}&limit={limit}
        
        List orders with optional filters.
        """
        orders = list(self._orders.values())
        
        # Apply filters
        if status:
            orders = [o for o in orders if o["status"] == status]
        if symbol:
            orders = [o for o in orders if o["symbol"] == symbol]
        
        # Sort by created_at descending
        orders.sort(key=lambda x: x["created_at"], reverse=True)
        
        # Apply limit
        orders = orders[:limit]
        
        return {
            "status": "success",
            "data": orders,
            "meta": {
                "total": len(orders),
                "limit": limit
            }
        }


# Demonstrate Order Management API
print("=" * 60)
print("ORDER MANAGEMENT API DEMONSTRATION")
print("=" * 60)

oms_api = OrderManagementAPI()

# 1. Submit order with idempotency key
print("\n1. Submit Order (with idempotency key):")
order1 = OrderRequest(
    symbol="AAPL",
    side=OrderSide.BUY,
    order_type=OrderType.LIMIT,
    quantity=100,
    price=150.00,
    client_order_id="client-001"
)
result1 = oms_api.submit_order(order1, idempotency_key="idem-key-001")
print(f"   Order ID: {result1['data']['order_id']}")
print(f"   Status: {result1['data']['status']}")

# 2. Submit same order again (idempotency check)
print("\n2. Retry Same Order (idempotency):")
result1_retry = oms_api.submit_order(order1, idempotency_key="idem-key-001")
print(f"   Idempotent Replay: {result1_retry['meta']['idempotent_replay']}")
print(f"   Same Order ID: {result1_retry['data']['order_id']}")

# 3. Get order
print("\n3. Get Order:")
order_id = result1['data']['order_id']
result_get = oms_api.get_order(order_id)
print(f"   Symbol: {result_get['data']['symbol']}")
print(f"   Status: {result_get['data']['status']}")

# 4. Modify order
print("\n4. Modify Order:")
modify_req = OrderModifyRequest(quantity=150, price=148.50)
result_modify = oms_api.modify_order(order_id, modify_req)
print(f"   New Quantity: {result_modify['data']['quantity']}")
print(f"   New Price: {result_modify['data']['price']}")

# 5. Cancel order
print("\n5. Cancel Order:")
result_cancel = oms_api.cancel_order(order_id)
print(f"   Status: {result_cancel['data']['status']}")
print(f"   Already Canceled: {result_cancel['meta']['already_canceled']}")

# 6. Idempotent cancel (already canceled)
print("\n6. Cancel Again (idempotent):")
result_cancel2 = oms_api.cancel_order(order_id)
print(f"   Already Canceled: {result_cancel2['meta']['already_canceled']}")

## 5. Rate Limiting and Throttling Implementation

### Rate Limiting Algorithms

1. **Token Bucket**: Smooth request rate with burst allowance
2. **Sliding Window**: Count requests in rolling time window
3. **Fixed Window**: Count requests in fixed time intervals
4. **Leaky Bucket**: Fixed rate output regardless of input

### Token Bucket Algorithm

```
         ┌─────────────────────────────┐
         │        TOKEN BUCKET         │
         │                             │
Tokens   │  ○ ○ ○ ○ ○ ○ ○ ○ ○ ○       │  Capacity: 10
Added    │                             │
per sec  │  Refill Rate: 5 tokens/sec  │
         │                             │
         └──────────────┬──────────────┘
                        │
                        ▼
                   Each Request
                   Consumes 1 Token
                        │
                        ▼
              ┌─────────────────────┐
              │ Token Available?    │
              │                     │
              │ YES → Allow Request │
              │ NO  → Rate Limited  │
              └─────────────────────┘
```

In [ ]:
# ==============================================================================
# Rate Limiting Implementations
# ==============================================================================

class TokenBucket:
    """
    Token Bucket rate limiter.
    
    Allows burst traffic up to bucket capacity, then limits
    to refill rate for sustained traffic.
    
    Commonly used by exchanges like Binance, FTX.
    """
    
    def __init__(
        self,
        capacity: int,
        refill_rate: float,
        refill_interval: float = 1.0
    ):
        """
        Args:
            capacity: Maximum tokens in bucket (burst allowance)
            refill_rate: Tokens added per refill_interval
            refill_interval: Time between refills in seconds
        """
        self.capacity = capacity
        self.refill_rate = refill_rate
        self.refill_interval = refill_interval
        self.tokens = capacity
        self.last_refill = time.time()
        self._lock = threading.Lock()
    
    def _refill(self):
        """Refill tokens based on elapsed time."""
        now = time.time()
        elapsed = now - self.last_refill
        
        # Calculate tokens to add
        tokens_to_add = (elapsed / self.refill_interval) * self.refill_rate
        
        self.tokens = min(self.capacity, self.tokens + tokens_to_add)
        self.last_refill = now
    
    def consume(self, tokens: int = 1) -> bool:
        """
        Attempt to consume tokens from bucket.
        
        Returns:
            True if tokens consumed, False if rate limited
        """
        with self._lock:
            self._refill()
            
            if self.tokens >= tokens:
                self.tokens -= tokens
                return True
            return False
    
    def get_status(self) -> Dict:
        """Get current bucket status."""
        with self._lock:
            self._refill()
            return {
                "tokens_available": round(self.tokens, 2),
                "capacity": self.capacity,
                "refill_rate": f"{self.refill_rate} tokens/{self.refill_interval}s"
            }


class SlidingWindowRateLimiter:
    """
    Sliding Window rate limiter.
    
    Counts requests in a rolling time window.
    More accurate than fixed window, prevents boundary burst.
    """
    
    def __init__(self, max_requests: int, window_seconds: float):
        """
        Args:
            max_requests: Maximum requests allowed in window
            window_seconds: Window size in seconds
        """
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.requests: deque = deque()
        self._lock = threading.Lock()
    
    def _cleanup_old_requests(self):
        """Remove requests outside the current window."""
        cutoff = time.time() - self.window_seconds
        while self.requests and self.requests[0] < cutoff:
            self.requests.popleft()
    
    def allow_request(self) -> bool:
        """
        Check if request is allowed.
        
        Returns:
            True if allowed, False if rate limited
        """
        with self._lock:
            self._cleanup_old_requests()
            
            if len(self.requests) < self.max_requests:
                self.requests.append(time.time())
                return True
            return False
    
    def get_status(self) -> Dict:
        """Get current rate limiter status."""
        with self._lock:
            self._cleanup_old_requests()
            return {
                "requests_used": len(self.requests),
                "max_requests": self.max_requests,
                "window_seconds": self.window_seconds,
                "remaining": self.max_requests - len(self.requests)
            }


class RateLimitedAPIClient:
    """
    API Client with built-in rate limiting.
    
    Demonstrates how to implement rate limiting on the client side
    to avoid hitting exchange rate limits.
    """
    
    def __init__(
        self,
        requests_per_second: int = 10,
        burst_capacity: int = 20
    ):
        self.rate_limiter = TokenBucket(
            capacity=burst_capacity,
            refill_rate=requests_per_second,
            refill_interval=1.0
        )
        self.request_count = 0
        self.rate_limited_count = 0
    
    def make_request(self, endpoint: str, weight: int = 1) -> Dict:
        """
        Make an API request with rate limiting.
        
        Args:
            endpoint: API endpoint
            weight: Request weight (some endpoints cost more)
        
        Returns:
            Response or rate limit error
        """
        if self.rate_limiter.consume(weight):
            self.request_count += 1
            # Simulate API call
            return {
                "success": True,
                "endpoint": endpoint,
                "request_number": self.request_count,
                "bucket_status": self.rate_limiter.get_status()
            }
        else:
            self.rate_limited_count += 1
            return {
                "success": False,
                "error": "RATE_LIMITED",
                "message": "Request rate limited, please retry later",
                "bucket_status": self.rate_limiter.get_status(),
                "retry_after_ms": 100  # Wait for tokens to refill
            }
    
    def get_stats(self) -> Dict:
        """Get request statistics."""
        return {
            "total_requests": self.request_count,
            "rate_limited": self.rate_limited_count,
            "success_rate": self.request_count / (self.request_count + self.rate_limited_count)
            if (self.request_count + self.rate_limited_count) > 0 else 0
        }


# Demonstrate Rate Limiting
print("=" * 60)
print("RATE LIMITING DEMONSTRATION")
print("=" * 60)

# Token Bucket Example
print("\n1. Token Bucket Rate Limiter:")
bucket = TokenBucket(capacity=5, refill_rate=2, refill_interval=1.0)
print(f"   Initial: {bucket.get_status()}")

# Simulate burst
print("\n   Simulating 7 rapid requests:")
for i in range(7):
    result = bucket.consume()
    status = bucket.get_status()
    print(f"   Request {i+1}: {'✓ Allowed' if result else '✗ Rate Limited'} "
          f"(tokens: {status['tokens_available']})")

# Wait and try again
print("\n   Waiting 1 second for refill...")
time.sleep(1)
print(f"   After wait: {bucket.get_status()}")

# Sliding Window Example
print("\n2. Sliding Window Rate Limiter:")
slider = SlidingWindowRateLimiter(max_requests=3, window_seconds=1.0)

print("   Simulating 5 rapid requests:")
for i in range(5):
    result = slider.allow_request()
    status = slider.get_status()
    print(f"   Request {i+1}: {'✓ Allowed' if result else '✗ Rate Limited'} "
          f"(used: {status['requests_used']}/{status['max_requests']})")

# Rate Limited Client Example
print("\n3. Rate Limited API Client:")
client = RateLimitedAPIClient(requests_per_second=5, burst_capacity=10)

print("   Making 15 rapid requests:")
for i in range(15):
    response = client.make_request(f"/api/quotes/AAPL")
    if response["success"]:
        print(f"   Request {i+1}: ✓ Success (tokens: {response['bucket_status']['tokens_available']:.1f})")
    else:
        print(f"   Request {i+1}: ✗ Rate Limited")

print(f"\n   Client Stats: {client.get_stats()}")

## 6. Authentication and API Key Management

### Authentication Methods in Trading APIs

1. **API Key + Secret**: Most common, used by Binance, Coinbase, etc.
2. **HMAC Signatures**: Sign requests with secret key
3. **OAuth 2.0**: Token-based authentication
4. **JWT**: JSON Web Tokens for stateless auth

### HMAC Signature Process

```
Request Parameters:
  timestamp=1641234567890
  symbol=AAPL
  quantity=100

1. Create message string:
   "timestamp=1641234567890&symbol=AAPL&quantity=100"

2. Sign with HMAC-SHA256:
   signature = HMAC-SHA256(secret_key, message)

3. Add to request headers:
   X-API-Key: your-api-key
   X-Signature: {signature}
   X-Timestamp: 1641234567890
```

In [ ]:
# ==============================================================================
# Authentication and Signature Implementation
# ==============================================================================

class HMACAuthenticator:
    """
    HMAC-based request authentication.
    
    Used by most major exchanges (Binance, Coinbase, Kraken, etc.)
    """
    
    def __init__(self, api_key: str, api_secret: str):
        self.api_key = api_key
        self.api_secret = api_secret.encode('utf-8')
    
    def create_signature(
        self,
        method: str,
        endpoint: str,
        params: Dict = None,
        body: Dict = None,
        timestamp: int = None
    ) -> Dict:
        """
        Create HMAC signature for a request.
        
        Returns headers to include in the request.
        """
        if timestamp is None:
            timestamp = int(time.time() * 1000)
        
        # Build message to sign
        message_parts = [
            str(timestamp),
            method.upper(),
            endpoint
        ]
        
        # Add query params
        if params:
            query_string = urlencode(sorted(params.items()))
            message_parts.append(query_string)
        
        # Add body
        if body:
            body_string = json.dumps(body, separators=(',', ':'), sort_keys=True)
            message_parts.append(body_string)
        
        message = ''.join(message_parts)
        
        # Create HMAC-SHA256 signature
        signature = hmac.new(
            self.api_secret,
            message.encode('utf-8'),
            hashlib.sha256
        ).hexdigest()
        
        return {
            "headers": {
                "X-API-Key": self.api_key,
                "X-Timestamp": str(timestamp),
                "X-Signature": signature
            },
            "message": message,
            "timestamp": timestamp
        }
    
    def verify_signature(
        self,
        received_signature: str,
        method: str,
        endpoint: str,
        params: Dict = None,
        body: Dict = None,
        timestamp: int = None
    ) -> bool:
        """
        Verify an incoming request signature (server-side).
        """
        expected = self.create_signature(method, endpoint, params, body, timestamp)
        return hmac.compare_digest(
            received_signature,
            expected["headers"]["X-Signature"]
        )


class APIKeyManager:
    """
    Secure API key management.
    
    Best Practices:
    - Never hardcode keys in source code
    - Use environment variables or secure vaults
    - Rotate keys regularly
    - Use separate keys for different environments
    """
    
    def __init__(self):
        self._keys: Dict[str, Dict] = {}
    
    def generate_key_pair(self, name: str, permissions: List[str] = None) -> Dict:
        """
        Generate a new API key pair.
        
        In production, use cryptographically secure generation.
        """
        api_key = f"pk_{secrets.token_hex(16)}"
        api_secret = secrets.token_hex(32)
        
        key_info = {
            "api_key": api_key,
            "api_secret": api_secret,
            "name": name,
            "permissions": permissions or ["read"],
            "created_at": datetime.now().isoformat(),
            "last_used": None,
            "rate_limit": 100,  # requests per minute
            "is_active": True
        }
        
        self._keys[api_key] = key_info
        
        return {
            "api_key": api_key,
            "api_secret": api_secret,  # Only shown once!
            "message": "Store the secret securely. It won't be shown again."
        }
    
    def validate_key(self, api_key: str) -> Dict:
        """Validate an API key and return its info."""
        if api_key not in self._keys:
            return {
                "valid": False,
                "error": "API key not found"
            }
        
        key_info = self._keys[api_key]
        
        if not key_info["is_active"]:
            return {
                "valid": False,
                "error": "API key is disabled"
            }
        
        # Update last used
        key_info["last_used"] = datetime.now().isoformat()
        
        return {
            "valid": True,
            "permissions": key_info["permissions"],
            "rate_limit": key_info["rate_limit"]
        }
    
    def revoke_key(self, api_key: str) -> Dict:
        """Revoke an API key."""
        if api_key not in self._keys:
            return {"success": False, "error": "Key not found"}
        
        self._keys[api_key]["is_active"] = False
        return {"success": True, "message": "Key revoked"}


class AuthenticatedAPIClient:
    """
    API client with built-in authentication.
    """
    
    def __init__(self, api_key: str, api_secret: str, base_url: str):
        self.base_url = base_url
        self.auth = HMACAuthenticator(api_key, api_secret)
    
    def _make_request(
        self,
        method: str,
        endpoint: str,
        params: Dict = None,
        body: Dict = None
    ) -> Dict:
        """
        Make an authenticated request.
        """
        # Get authentication headers
        auth_result = self.auth.create_signature(method, endpoint, params, body)
        headers = auth_result["headers"]
        
        # In production: use requests library
        # response = requests.request(
        #     method=method,
        #     url=f"{self.base_url}{endpoint}",
        #     headers=headers,
        #     params=params,
        #     json=body
        # )
        
        # Simulated response
        return {
            "success": True,
            "request": {
                "method": method,
                "endpoint": endpoint,
                "headers": headers,
                "message_signed": auth_result["message"][:50] + "..."
            }
        }
    
    def get_account_balance(self) -> Dict:
        """GET /v1/account/balance"""
        return self._make_request("GET", "/v1/account/balance")
    
    def submit_order(self, order: Dict) -> Dict:
        """POST /v1/orders"""
        return self._make_request("POST", "/v1/orders", body=order)


# Demonstrate Authentication
print("=" * 60)
print("AUTHENTICATION DEMONSTRATION")
print("=" * 60)

# 1. Generate API Keys
print("\n1. Generate API Key Pair:")
key_manager = APIKeyManager()
key_pair = key_manager.generate_key_pair(
    name="trading-bot",
    permissions=["read", "trade"]
)
print(f"   API Key: {key_pair['api_key']}")
print(f"   API Secret: {key_pair['api_secret'][:20]}...")
print(f"   Message: {key_pair['message']}")

# 2. Create HMAC Signature
print("\n2. HMAC Signature Creation:")
auth = HMACAuthenticator(key_pair["api_key"], key_pair["api_secret"])

# Sign a GET request
sig_result = auth.create_signature(
    method="GET",
    endpoint="/v1/account/balance",
    params={"currency": "USD"}
)
print(f"   Headers to send:")
for key, value in sig_result["headers"].items():
    print(f"     {key}: {value[:40]}...")
print(f"   Message signed: {sig_result['message']}")

# Sign a POST request
print("\n3. Sign POST Request (Order Submission):")
order_body = {
    "symbol": "AAPL",
    "side": "buy",
    "quantity": 100,
    "price": 150.00
}
sig_result_post = auth.create_signature(
    method="POST",
    endpoint="/v1/orders",
    body=order_body
)
print(f"   Signature: {sig_result_post['headers']['X-Signature'][:40]}...")

# 4. Verify Signature (Server Side)
print("\n4. Server-side Signature Verification:")
is_valid = auth.verify_signature(
    received_signature=sig_result_post["headers"]["X-Signature"],
    method="POST",
    endpoint="/v1/orders",
    body=order_body,
    timestamp=sig_result_post["timestamp"]
)
print(f"   Signature Valid: {'✓ Yes' if is_valid else '✗ No'}")

# 5. Authenticated Client Usage
print("\n5. Authenticated API Client:")
client = AuthenticatedAPIClient(
    api_key=key_pair["api_key"],
    api_secret=key_pair["api_secret"],
    base_url="https://api.trading.example.com"
)

balance_response = client.get_account_balance()
print(f"   Balance Request Headers: {balance_response['request']['headers']['X-API-Key'][:20]}...")

## 7. Request/Response Models with Pydantic

### Why Pydantic for Trading APIs?

1. **Type Safety**: Catch errors at validation time, not runtime
2. **Documentation**: Auto-generate OpenAPI schemas
3. **Serialization**: Easy JSON conversion
4. **Validation**: Built-in validators for complex rules
5. **Performance**: Fast validation using Cython

### Model Hierarchy

```
BaseModel
    ├── QuoteResponse
    ├── CandleResponse
    ├── OrderRequest
    │   ├── MarketOrderRequest
    │   └── LimitOrderRequest
    ├── OrderResponse
    ├── PositionResponse
    └── AccountResponse
```

In [ ]:
# ==============================================================================
# Comprehensive Pydantic Models for Trading API
# ==============================================================================

# Base Response Wrapper
class APIResponse(BaseModel):
    """Standard API response wrapper."""
    success: bool
    request_id: str = Field(default_factory=lambda: str(uuid.uuid4()))
    timestamp: datetime = Field(default_factory=datetime.now)
    
    class Config:
        json_encoders = {
            datetime: lambda v: v.isoformat()
        }


class ErrorDetail(BaseModel):
    """Error detail model."""
    code: str
    message: str
    field: Optional[str] = None


class ErrorResponse(APIResponse):
    """Error response model."""
    success: bool = False
    error: ErrorDetail


# Market Data Models
class Quote(BaseModel):
    """Real-time quote model."""
    symbol: str
    bid: float = Field(..., ge=0)
    ask: float = Field(..., ge=0)
    bid_size: int = Field(..., ge=0)
    ask_size: int = Field(..., ge=0)
    last: float = Field(..., ge=0)
    volume: int = Field(..., ge=0)
    timestamp: datetime
    
    @property
    def spread(self) -> float:
        """Calculate bid-ask spread."""
        return self.ask - self.bid
    
    @property
    def spread_bps(self) -> float:
        """Calculate spread in basis points."""
        mid = (self.bid + self.ask) / 2
        return (self.spread / mid) * 10000 if mid > 0 else 0


class QuoteResponse(APIResponse):
    """Quote API response."""
    success: bool = True
    data: Quote


class OHLCV(BaseModel):
    """OHLCV candle model."""
    timestamp: datetime
    open: float = Field(..., ge=0)
    high: float = Field(..., ge=0)
    low: float = Field(..., ge=0)
    close: float = Field(..., ge=0)
    volume: int = Field(..., ge=0)
    
    @validator('high')
    def high_must_be_highest(cls, v, values):
        """Validate high is the highest price."""
        if 'open' in values and v < values['open']:
            raise ValueError('high must be >= open')
        if 'low' in values and v < values['low']:
            raise ValueError('high must be >= low')
        return v
    
    @validator('low')
    def low_must_be_lowest(cls, v, values):
        """Validate low is the lowest price."""
        if 'open' in values and v > values['open']:
            raise ValueError('low must be <= open')
        return v


class CandlesResponse(APIResponse):
    """Candles API response."""
    success: bool = True
    data: List[OHLCV]
    symbol: str
    interval: str


# Position and Account Models
class Position(BaseModel):
    """Trading position model."""
    symbol: str
    quantity: float
    side: str = Field(..., regex="^(long|short)$")
    entry_price: float = Field(..., gt=0)
    current_price: float = Field(..., gt=0)
    unrealized_pnl: float
    realized_pnl: float = 0.0
    opened_at: datetime
    
    @property
    def pnl_percent(self) -> float:
        """Calculate P&L percentage."""
        return ((self.current_price - self.entry_price) / self.entry_price) * 100


class Balance(BaseModel):
    """Account balance model."""
    currency: str
    total: float = Field(..., ge=0)
    available: float = Field(..., ge=0)
    locked: float = Field(..., ge=0)
    
    @root_validator
    def validate_balance(cls, values):
        """Validate balance components add up."""
        total = values.get('total', 0)
        available = values.get('available', 0)
        locked = values.get('locked', 0)
        
        if abs(total - (available + locked)) > 0.01:  # Allow small floating point error
            raise ValueError('total must equal available + locked')
        return values


class AccountResponse(APIResponse):
    """Account API response."""
    success: bool = True
    balances: List[Balance]
    positions: List[Position] = []
    margin_used: float = 0.0
    margin_available: float = 0.0


# Demonstrate Pydantic Models
print("=" * 60)
print("PYDANTIC MODEL EXAMPLES")
print("=" * 60)

# 1. Quote Model
print("\n1. Quote Model:")
quote = Quote(
    symbol="AAPL",
    bid=150.25,
    ask=150.27,
    bid_size=500,
    ask_size=300,
    last=150.26,
    volume=1000000,
    timestamp=datetime.now()
)
print(f"   Symbol: {quote.symbol}")
print(f"   Bid/Ask: ${quote.bid} / ${quote.ask}")
print(f"   Spread: ${quote.spread:.4f} ({quote.spread_bps:.2f} bps)")
print(f"   JSON: {quote.json()[:80]}...")

# 2. OHLCV Validation
print("\n2. OHLCV Validation:")
try:
    valid_candle = OHLCV(
        timestamp=datetime.now(),
        open=150.00,
        high=152.00,
        low=149.00,
        close=151.00,
        volume=100000
    )
    print(f"   ✓ Valid candle: O={valid_candle.open}, H={valid_candle.high}, "
          f"L={valid_candle.low}, C={valid_candle.close}")
except ValidationError as e:
    print(f"   ✗ Error: {e}")

# Invalid candle (high < low)
print("\n3. Invalid OHLCV (high < low):")
try:
    invalid_candle = OHLCV(
        timestamp=datetime.now(),
        open=150.00,
        high=148.00,  # Invalid: lower than low
        low=149.00,
        close=151.00,
        volume=100000
    )
except ValidationError as e:
    print(f"   ✗ Validation Error: {e.errors()[0]['msg']}")

# 3. Balance Validation
print("\n4. Balance Validation:")
try:
    balance = Balance(
        currency="USD",
        total=100000.00,
        available=85000.00,
        locked=15000.00
    )
    print(f"   ✓ Valid: Total=${balance.total:,.2f}, "
          f"Available=${balance.available:,.2f}, Locked=${balance.locked:,.2f}")
except ValidationError as e:
    print(f"   ✗ Error: {e}")

# Invalid balance (doesn't add up)
print("\n5. Invalid Balance (doesn't add up):")
try:
    invalid_balance = Balance(
        currency="USD",
        total=100000.00,
        available=90000.00,
        locked=15000.00  # 90k + 15k != 100k
    )
except ValidationError as e:
    print(f"   ✗ Validation Error: {e.errors()[0]['msg']}")

# 4. Position with P&L
print("\n6. Position Model:")
position = Position(
    symbol="AAPL",
    quantity=100,
    side="long",
    entry_price=145.00,
    current_price=150.00,
    unrealized_pnl=500.00,
    opened_at=datetime.now() - timedelta(days=5)
)
print(f"   Symbol: {position.symbol}")
print(f"   Entry: ${position.entry_price:.2f}, Current: ${position.current_price:.2f}")
print(f"   P&L: ${position.unrealized_pnl:.2f} ({position.pnl_percent:.2f}%)")

## 8. Async API Client Implementation

### Why Async for Trading Systems?

1. **Concurrent Requests**: Fetch data from multiple sources simultaneously
2. **Non-blocking I/O**: Don't wait for slow network operations
3. **WebSocket Handling**: Native async support for real-time data
4. **Scalability**: Handle many connections with minimal resources

### Async Patterns in Trading

```python
# Concurrent market data fetching
async def fetch_all_quotes(symbols):
    tasks = [fetch_quote(s) for s in symbols]
    return await asyncio.gather(*tasks)

# Connection pooling
async with aiohttp.ClientSession() as session:
    # Reuse connection for multiple requests
    pass
```

In [ ]:
# ==============================================================================
# Async Trading API Client
# ==============================================================================

class AsyncTradingClient:
    """
    Async trading API client with connection pooling and concurrent requests.
    
    This is a simulation - in production, use aiohttp:
    
    ```python
    import aiohttp
    
    async def fetch(session, url):
        async with session.get(url) as response:
            return await response.json()
    ```
    """
    
    def __init__(
        self,
        base_url: str,
        api_key: str,
        api_secret: str,
        max_connections: int = 10,
        timeout: float = 30.0
    ):
        self.base_url = base_url
        self.auth = HMACAuthenticator(api_key, api_secret)
        self.max_connections = max_connections
        self.timeout = timeout
        self._session = None
        self._request_count = 0
    
    async def _simulate_request(self, endpoint: str, latency: float = 0.01) -> Dict:
        """Simulate async HTTP request with artificial latency."""
        await asyncio.sleep(latency + np.random.uniform(0, 0.01))
        self._request_count += 1
        return {
            "success": True,
            "endpoint": endpoint,
            "request_number": self._request_count
        }
    
    async def get_quote(self, symbol: str) -> Dict:
        """
        Async GET /v1/quotes/{symbol}
        """
        result = await self._simulate_request(f"/v1/quotes/{symbol}")
        
        # Generate mock quote
        np.random.seed(hash(symbol) % 2**32)
        base_price = 100 + hash(symbol) % 400
        
        return {
            **result,
            "data": {
                "symbol": symbol,
                "bid": round(base_price * 0.999, 2),
                "ask": round(base_price * 1.001, 2),
                "last": round(base_price, 2),
                "timestamp": datetime.now().isoformat()
            }
        }
    
    async def get_multiple_quotes(self, symbols: List[str]) -> List[Dict]:
        """
        Fetch quotes for multiple symbols concurrently.
        
        This demonstrates the power of async - all requests run in parallel.
        """
        tasks = [self.get_quote(symbol) for symbol in symbols]
        return await asyncio.gather(*tasks)
    
    async def get_candles(
        self,
        symbol: str,
        interval: str = "1h",
        limit: int = 100
    ) -> Dict:
        """
        Async GET /v1/candles/{symbol}
        """
        await self._simulate_request(f"/v1/candles/{symbol}")
        
        # Generate mock candles
        np.random.seed(hash(symbol) % 2**32)
        base_price = 100 + hash(symbol) % 400
        
        candles = []
        for i in range(limit):
            timestamp = datetime.now() - timedelta(hours=limit - i)
            open_price = base_price * (1 + np.random.uniform(-0.01, 0.01))
            close_price = open_price * (1 + np.random.uniform(-0.005, 0.005))
            
            candles.append({
                "timestamp": timestamp.isoformat(),
                "open": round(open_price, 2),
                "high": round(max(open_price, close_price) * 1.005, 2),
                "low": round(min(open_price, close_price) * 0.995, 2),
                "close": round(close_price, 2),
                "volume": np.random.randint(10000, 100000)
            })
            base_price = close_price
        
        return {
            "success": True,
            "data": {
                "symbol": symbol,
                "interval": interval,
                "candles": candles
            }
        }
    
    async def submit_order(self, order: Dict) -> Dict:
        """
        Async POST /v1/orders
        """
        await self._simulate_request("/v1/orders", latency=0.05)
        
        return {
            "success": True,
            "data": {
                "order_id": f"ORD-{uuid.uuid4().hex[:12].upper()}",
                **order,
                "status": "open",
                "created_at": datetime.now().isoformat()
            }
        }
    
    async def get_account_balance(self) -> Dict:
        """
        Async GET /v1/account/balance
        """
        await self._simulate_request("/v1/account/balance")
        
        return {
            "success": True,
            "data": {
                "balances": [
                    {"currency": "USD", "total": 100000.00, "available": 85000.00},
                    {"currency": "BTC", "total": 2.5, "available": 2.0},
                    {"currency": "ETH", "total": 10.0, "available": 8.5}
                ]
            }
        }


async def demonstrate_async_client():
    """Demonstrate async client capabilities."""
    print("=" * 60)
    print("ASYNC API CLIENT DEMONSTRATION")
    print("=" * 60)
    
    # Create client
    client = AsyncTradingClient(
        base_url="https://api.trading.example.com",
        api_key="demo-key",
        api_secret="demo-secret"
    )
    
    # 1. Sequential vs Concurrent comparison
    symbols = ["AAPL", "GOOGL", "MSFT", "TSLA", "AMZN", "META", "NVDA", "AMD"]
    
    # Sequential (for comparison)
    print("\n1. Sequential Quotes (simulated):")
    start_seq = time.time()
    sequential_results = []
    for symbol in symbols:
        result = await client.get_quote(symbol)
        sequential_results.append(result)
    seq_time = time.time() - start_seq
    print(f"   Time: {seq_time:.3f}s for {len(symbols)} symbols")
    
    # Concurrent
    print("\n2. Concurrent Quotes:")
    start_conc = time.time()
    concurrent_results = await client.get_multiple_quotes(symbols)
    conc_time = time.time() - start_conc
    print(f"   Time: {conc_time:.3f}s for {len(symbols)} symbols")
    print(f"   Speedup: {seq_time / conc_time:.1f}x faster")
    
    # Show results
    print("\n   Results:")
    for result in concurrent_results[:3]:
        data = result["data"]
        print(f"   {data['symbol']}: ${data['last']:.2f}")
    print("   ...")
    
    # 3. Multiple parallel operations
    print("\n3. Parallel Operations (quotes + candles + balance):")
    start_parallel = time.time()
    
    # Run different operations in parallel
    quote_task = client.get_quote("AAPL")
    candles_task = client.get_candles("AAPL", limit=10)
    balance_task = client.get_account_balance()
    
    quote, candles, balance = await asyncio.gather(
        quote_task,
        candles_task,
        balance_task
    )
    
    parallel_time = time.time() - start_parallel
    print(f"   Time: {parallel_time:.3f}s for 3 different operations")
    print(f"   Quote: {quote['data']['symbol']} @ ${quote['data']['last']}")
    print(f"   Candles: {len(candles['data']['candles'])} data points")
    print(f"   Balance: ${balance['data']['balances'][0]['total']:,.2f} USD")
    
    # 4. Batch order submission
    print("\n4. Concurrent Order Submission:")
    orders = [
        {"symbol": "AAPL", "side": "buy", "quantity": 100, "price": 150.00},
        {"symbol": "GOOGL", "side": "sell", "quantity": 50, "price": 140.00},
        {"symbol": "MSFT", "side": "buy", "quantity": 75, "price": 380.00}
    ]
    
    start_orders = time.time()
    order_tasks = [client.submit_order(order) for order in orders]
    order_results = await asyncio.gather(*order_tasks)
    orders_time = time.time() - start_orders
    
    print(f"   Submitted {len(order_results)} orders in {orders_time:.3f}s")
    for result in order_results:
        print(f"   Order {result['data']['order_id']}: {result['data']['symbol']}")
    
    print(f"\n   Total API Requests: {client._request_count}")


# Run the async demonstration
await demonstrate_async_client()

## 9. Error Handling and Retry Logic

### Error Categories in Trading APIs

1. **Transient Errors** (should retry):
   - Network timeouts
   - Rate limiting (429)
   - Server overload (503)

2. **Permanent Errors** (should NOT retry):
   - Authentication failure (401)
   - Invalid request (400)
   - Resource not found (404)

### Exponential Backoff with Jitter

```
Retry 1: wait 1s  + random(0, 0.5)s
Retry 2: wait 2s  + random(0, 1.0)s
Retry 3: wait 4s  + random(0, 2.0)s
Retry 4: wait 8s  + random(0, 4.0)s
...
```

Adding jitter prevents "thundering herd" when multiple clients retry simultaneously.

In [ ]:
# ==============================================================================
# Error Handling and Retry Logic
# ==============================================================================

class APIError(Exception):
    """Base API error."""
    def __init__(self, code: str, message: str, status_code: int = 500, retryable: bool = False):
        self.code = code
        self.message = message
        self.status_code = status_code
        self.retryable = retryable
        super().__init__(message)


class RateLimitError(APIError):
    """Rate limit exceeded error."""
    def __init__(self, retry_after: int = 60):
        super().__init__(
            code="RATE_LIMIT_EXCEEDED",
            message=f"Rate limit exceeded. Retry after {retry_after} seconds.",
            status_code=429,
            retryable=True
        )
        self.retry_after = retry_after


class AuthenticationError(APIError):
    """Authentication failed error."""
    def __init__(self, message: str = "Authentication failed"):
        super().__init__(
            code="AUTHENTICATION_FAILED",
            message=message,
            status_code=401,
            retryable=False  # Don't retry auth failures
        )


class NetworkError(APIError):
    """Network-related error."""
    def __init__(self, message: str = "Network error"):
        super().__init__(
            code="NETWORK_ERROR",
            message=message,
            status_code=0,
            retryable=True
        )


class RetryStrategy:
    """
    Configurable retry strategy with exponential backoff.
    """
    
    def __init__(
        self,
        max_retries: int = 3,
        base_delay: float = 1.0,
        max_delay: float = 60.0,
        exponential_base: float = 2.0,
        jitter: bool = True
    ):
        self.max_retries = max_retries
        self.base_delay = base_delay
        self.max_delay = max_delay
        self.exponential_base = exponential_base
        self.jitter = jitter
    
    def calculate_delay(self, attempt: int) -> float:
        """
        Calculate delay for given attempt number (0-indexed).
        
        Uses exponential backoff with optional jitter.
        """
        # Exponential backoff
        delay = self.base_delay * (self.exponential_base ** attempt)
        
        # Cap at max delay
        delay = min(delay, self.max_delay)
        
        # Add jitter (random factor 0.5 to 1.5)
        if self.jitter:
            jitter_factor = 0.5 + np.random.random()
            delay *= jitter_factor
        
        return delay
    
    def get_delays(self) -> List[float]:
        """Get all retry delays for visualization."""
        return [self.calculate_delay(i) for i in range(self.max_retries)]


class ResilientAPIClient:
    """
    API client with built-in retry logic and error handling.
    """
    
    def __init__(
        self,
        base_url: str,
        retry_strategy: RetryStrategy = None
    ):
        self.base_url = base_url
        self.retry_strategy = retry_strategy or RetryStrategy()
        self._stats = {
            "requests": 0,
            "successes": 0,
            "retries": 0,
            "failures": 0
        }
    
    def _should_retry(self, error: Exception, attempt: int) -> bool:
        """Determine if request should be retried."""
        if attempt >= self.retry_strategy.max_retries:
            return False
        
        if isinstance(error, APIError):
            return error.retryable
        
        # Retry on generic exceptions (network issues)
        return True
    
    def _simulate_request(self, endpoint: str, fail_rate: float = 0.3) -> Dict:
        """
        Simulate a request that may fail.
        
        Args:
            endpoint: API endpoint
            fail_rate: Probability of failure (for demonstration)
        """
        self._stats["requests"] += 1
        
        # Simulate various error conditions
        if np.random.random() < fail_rate:
            error_type = np.random.choice([
                "rate_limit", "network", "server_error"
            ], p=[0.3, 0.4, 0.3])
            
            if error_type == "rate_limit":
                raise RateLimitError(retry_after=5)
            elif error_type == "network":
                raise NetworkError("Connection timeout")
            else:
                raise APIError(
                    code="SERVER_ERROR",
                    message="Internal server error",
                    status_code=500,
                    retryable=True
                )
        
        return {
            "success": True,
            "data": {"endpoint": endpoint, "timestamp": datetime.now().isoformat()}
        }
    
    def request_with_retry(
        self,
        endpoint: str,
        fail_rate: float = 0.3
    ) -> Dict:
        """
        Make request with automatic retry on failure.
        """
        last_error = None
        attempt_log = []
        
        for attempt in range(self.retry_strategy.max_retries + 1):
            try:
                result = self._simulate_request(endpoint, fail_rate)
                self._stats["successes"] += 1
                
                return {
                    **result,
                    "attempts": attempt + 1,
                    "attempt_log": attempt_log
                }
                
            except Exception as e:
                last_error = e
                
                if not self._should_retry(e, attempt):
                    break
                
                delay = self.retry_strategy.calculate_delay(attempt)
                
                attempt_log.append({
                    "attempt": attempt + 1,
                    "error": str(e),
                    "delay": round(delay, 3),
                    "retrying": True
                })
                
                self._stats["retries"] += 1
                
                # In production: await asyncio.sleep(delay)
                time.sleep(delay * 0.1)  # Reduced delay for demo
        
        self._stats["failures"] += 1
        
        return {
            "success": False,
            "error": {
                "code": last_error.code if isinstance(last_error, APIError) else "UNKNOWN",
                "message": str(last_error)
            },
            "attempts": len(attempt_log) + 1,
            "attempt_log": attempt_log
        }
    
    def get_stats(self) -> Dict:
        """Get request statistics."""
        total = self._stats["successes"] + self._stats["failures"]
        return {
            **self._stats,
            "success_rate": self._stats["successes"] / total if total > 0 else 0
        }


# Demonstrate Error Handling
print("=" * 60)
print("ERROR HANDLING AND RETRY DEMONSTRATION")
print("=" * 60)

# 1. Retry Strategy Delays
print("\n1. Exponential Backoff Delays:")
strategy = RetryStrategy(max_retries=5, base_delay=1.0, jitter=False)
print(f"   Without jitter: {[f'{d:.1f}s' for d in strategy.get_delays()]}")

strategy_jitter = RetryStrategy(max_retries=5, base_delay=1.0, jitter=True)
print(f"   With jitter:    {[f'{d:.1f}s' for d in strategy_jitter.get_delays()]}")

# 2. Resilient Client Demo
print("\n2. Resilient API Client:")
np.random.seed(42)
client = ResilientAPIClient(
    base_url="https://api.trading.example.com",
    retry_strategy=RetryStrategy(max_retries=3, base_delay=0.1)
)

# Make several requests with high failure rate
print("\n   Making 10 requests (50% simulated failure rate):")
for i in range(10):
    result = client.request_with_retry("/v1/quotes/AAPL", fail_rate=0.5)
    
    if result["success"]:
        attempts = result["attempts"]
        print(f"   Request {i+1}: ✓ Success (attempts: {attempts})")
    else:
        print(f"   Request {i+1}: ✗ Failed after {result['attempts']} attempts - {result['error']['code']}")
        if result["attempt_log"]:
            for log in result["attempt_log"][:2]:
                print(f"      Retry {log['attempt']}: {log['error'][:30]}... (wait {log['delay']}s)")

# 3. Statistics
print(f"\n3. Client Statistics:")
stats = client.get_stats()
print(f"   Total Requests: {stats['requests']}")
print(f"   Successes: {stats['successes']}")
print(f"   Retries: {stats['retries']}")
print(f"   Final Failures: {stats['failures']}")
print(f"   Success Rate: {stats['success_rate']:.1%}")

# 4. Error Classification
print("\n4. Error Classification:")
errors = [
    RateLimitError(retry_after=30),
    AuthenticationError("Invalid API key"),
    NetworkError("Connection refused"),
    APIError("INVALID_ORDER", "Order quantity too small", 400, False)
]

for error in errors:
    retry_text = "✓ Retry" if error.retryable else "✗ No Retry"
    print(f"   {error.code}: {retry_text} (HTTP {error.status_code})")

## 10. API Versioning Strategies

### Versioning Approaches

| Strategy | Example | Pros | Cons |
|----------|---------|------|------|
| URL Path | `/v1/orders` | Clear, easy routing | URL changes |
| Query Param | `/orders?version=1` | URL stays same | Easy to miss |
| Header | `Accept: application/vnd.api.v1+json` | Clean URLs | Hidden, harder to test |

### Best Practices

1. **Use semantic versioning**: Major.Minor.Patch
2. **Deprecation notices**: Warn before removing features
3. **Backward compatibility**: Don't break existing clients
4. **Sunset headers**: Include deprecation timeline